In [1]:
import json
import re
import os
from tika import parser

In [2]:
resultVorgang='resultVorgangAll.json'
with open(resultVorgang) as f:
    resultVorgang = json.load(f)

In [3]:
#target_dir = r"C:\Users\schull\Projekte\KIbarDok\testDatenMerge"
target_dir = r"C:\Users\guendm1\Documents\BerlinDenkmal\Testdaten\All\doc"
os.chdir(target_dir)
files = sorted(os.listdir(target_dir))

In [4]:
def remManContent(inhalt):
    inhalt = inhalt.replace("\n", " ")
    inhalt=' '.join(inhalt.split())
    #########################################################################################################stop looking in unnecessary content
    searchAuflage = re.search('Auflage[n]*\sund\sBedingung[en]*|Auflage[n]*[^.]*[:]',inhalt)
    if searchAuflage:
        startInhalt=searchAuflage.span()[1]#[1] damit Auflafe auch im Match berücksichtigt wird.
        inhalt=inhalt[:startInhalt]
    else:
        searchSachverhalt=re.search('Begründung.*Sachverhalt',inhalt)
        if searchSachverhalt:
            startInhalt=searchSachverhalt.span()[0]
            inhalt=inhalt[:startInhalt]
        else:
            searchAusgleich=re.search('Entscheidung\süber\sden\sAusgleich\sin\sGeld',inhalt)
            if searchAusgleich:
                startRemove=searchAusgleich.span()[0]
                inhalt=inhalt[:startRemove]
    #################################################remove "Ausgleich in Geld ist nicht genehmigungsfähig"
    search=re.search('(Ausgleich.+Geld.+gewähren)', inhalt)
    if search:
        startInhalt=search.span()[0]
        stopInhalt=search.span()[1]
        removeInhalt=inhalt[startInhalt:stopInhalt]
        inhalt=inhalt.replace(removeInhalt,'')
    ########################################################################################################
    return inhalt

In [25]:
#for file in files:
result={}
result['Anhörung']=[]
result['Genehmigung']=[]
result['Versagung']=[]
result['Nachforderung']=[]
result['Bauverfahren']=[]
for i in range(0,len(files)):
    file=files[i]
    #print('file: ', file)
    raw = parser.from_file(target_dir+'\\'+file)
    inhalt = raw['content']
    if inhalt == None or inhalt =='':
        #print("File: empty:", file)
        continue
    #inhalt=remManContent(inhalt)
    find=re.findall('[^.]*[\d+.\d+.+\d+]*[GVBL.]*[S.]*[^.]*[.][^.]*[§][\s]*\d+\sAbs.[^.]*[\d+.\d+.+\d+]*[GVBL.]*[S.]*[^.]*[.][^.]*',inhalt)
    findPara=re.findall('[§][\s]*\d+\sAbs.[\s]*\d+',inhalt)
    if find:
        if file in resultVorgang['Gesamt-Zuordnung'].keys():
            #print('keys',resultVorgang['Gesamt-Zuordnung'][file])
            vorgang=resultVorgang['Gesamt-Zuordnung'][file]
            if ('Antrag' or 'Anfrage') not in vorgang.keys():
                print(i,'--', file, '--',vorgang,'--',findPara)
                print('text: ', find)
                result[file]={}
                result[file]['Vorgang']=list(set(list(vorgang.keys())))
                result[file]['Gesetz']=list(set(findPara))
                result[file]['Passage']=find#[str(elFind.encode("utf-8")) for elFind in find]
                if 'Genehmigung' in vorgang.keys():
                    result['Genehmigung'].extend(list(set(findPara)))
                elif 'Anhörung' in vorgang.keys():
                    result['Anhörung'].extend(list(set(findPara)))
                elif 'Versagung' in vorgang.keys():
                    result['Versagung'].extend(list(set(findPara)))
                elif 'Bauverfahren' in vorgang.keys():
                    result['Bauverfahren'].extend(list(set(findPara)))
                elif 'Nachforderung' in vorgang.keys():
                    result['Nachforderung'].extend(list(set(findPara)))
        elif file in resultVorgang['Keine-Zuordnung'].keys():
            vorgang='Keine Zuordnung'
        else:
            vorgang='Informelles Format'
#     if i==50:
#         break

16 -- 1mündliche BVV-Anfrage_Abriss A8-Geb. Ziesel-Bau.doc -- {'Anfrage': ['Dateiname']} -- ['§ 6 Abs. 5', '§ 6 Abs. 5']
text:  [' 12. 2006  \nBetreff Abriss Gebäude A 8 – Ziesel-Bau\n\nFrage:\n\nEntfaltete das Bezirksamt nach seiner Wahl am 26. Oktober 2006 Aktivitäten, mit denen es seinen Einfluss auf den Abriss des als Ziesel-Bau bekannten Fabrikgebäudes in Oberschöneweide geltend zu machen versuchte ?\n\nAntwort:\n\nDie Entscheidung für die Beseitigung des Gebäudes A 8 (Fernmeldekabelfabrik) wurde gemäß § 6 Abs. 5 DSchG Bln ( Denkmalschutzgesetz Berlin) von der Obersten Denkmalschutzbehörde mit Datum vom 01. 02. 2006 getroffen', ' 12. 2006  \nBetreff Abriss Gebäude A 8 – Ziesel-Bau\n\nFrage:\n\nEntfaltete das Bezirksamt nach seiner Wahl am 26. Oktober 2006 Aktivitäten, mit denen es seinen Einfluss auf den Abriss des als Ziesel-Bau bekannten Fabrikgebäudes in Oberschöneweide geltend zu machen versuchte ?\n\nAntwort:\n\nDie Entscheidung für die Beseitigung des Gebäudes A 8 (Fernmelde

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\guendm1\\Documents\\BerlinDenkmal\\Testdaten\\All\\doc\\~$rsagung Bescheid.doc'

In [26]:
result['Genehmigung']=list(set(result['Genehmigung']))
result['Anhörung']=list(set(result['Anhörung']))
result['Bauverfahren']=list(set(result['Bauverfahren']))
result['Versagung']=list(set(result['Versagung']))
result['Nachforderung']=list(set(result['Nachforderung']))

In [27]:
#save objNrValue as json file
path=r'C:\Users\guendm1\Documents\BerlinDenkmal\Denkmalliste\Vorgang\\'
with open(path+'resultVorgangParagraphUncut.json', 'w', encoding='utf8') as fp:
    json.dump(result, fp,  indent=4,ensure_ascii=False)
print('successfully saved')

successfully saved
